Task0

In [52]:
from jedi.api.refactoring import inline
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Task1

In [53]:
import pandas as pd
frame = pd.DataFrame(pd.read_csv("./mammographic_masses_data.csv"))
print(frame)

      BA   Age  Shape  Margin  Density  Severity
0    5.0  67.0    3.0     5.0      3.0         1
1    4.0  43.0    1.0     1.0      NaN         1
2    5.0  58.0    4.0     5.0      3.0         1
3    4.0  28.0    1.0     1.0      3.0         0
4    5.0  74.0    1.0     5.0      NaN         1
..   ...   ...    ...     ...      ...       ...
956  4.0  47.0    2.0     1.0      3.0         0
957  4.0  56.0    4.0     5.0      3.0         1
958  4.0  64.0    4.0     5.0      3.0         0
959  5.0  66.0    4.0     5.0      3.0         1
960  4.0  62.0    3.0     3.0      3.0         0

[961 rows x 6 columns]


In [ ]:
import plotly.express as px

#TODO 2.1: Is de basic funtie van summary in dataframe
# One important function of pandas is to fill in missing values.
# df.fillna(0) will replace all NaN values with 0.
print(frame.info())

#2.2
# Makes a histogram of the Margin values of the rows in the database where Severity = 1
fig = px.histogram(frame.loc[frame.Severity == 1].Margin, x="Margin")
fig.show()

#2.3
fig = px.bar(frame, x="Age", y="Severity", color="Severity", title="Long-Form Input")
fig.show()

fig = px.density_heatmap(frame, x="Shape", y="Severity", marginal_x="histogram", marginal_y="histogram")
fig.show()

fig = px.scatter(frame, x="BA", y="Age", color="Severity")
fig.show()

NameError: name 'frame' is not defined

Task3 - NOT FINISHED

In [55]:
copy = frame.copy()

# 3.1
copy = copy.dropna()

fig = px.histogram(copy.loc[copy.Severity == 1].Margin, x="Margin")
fig.show()

# 3.2


Task4 - NOT FINISHED

In [64]:
from sklearn.feature_selection import VarianceThreshold

# 4.1
X = frame.copy()
sel = VarianceThreshold(threshold=(.5 * (1 - .5))) # betere threshold kiezen & kijken hoe je iets anders dan 0 en 1 verwijderd
sel.fit_transform(X)

fig = px.histogram(X.loc[frame.Severity == 1].Margin, x="Margin")
fig.show()

# 4.2 - volgensmij af? We moeten alleen nog even kijken of het klopt
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
components = pca.fit_transform(copy)

fig = px.scatter(components, x=0, y=1, color=copy['Margin'])
fig.show()